In [1]:
import numpy as np
import matplotlib.pyplot as plt
from idea import IDEA

In [2]:
def g1_objective(x):
    c = np.stack([
        # objective
        5 * x[:, :4].sum(1) - 5 * (x[:, :4]**2).sum(1) - x[:, 4:].sum(1),
        # constraints
        2*x[:, 0] + 2*x[:, 1] + x[:, 9] + x[:, 10] - 10.,
        2*x[:, 0] + 2*x[:, 2] + x[:, 9] + x[:, 11] - 10.,
        2*x[:, 1] + 2*x[:, 2] + x[:, 10] + x[:, 11] - 10.,
        -8*x[:, 0] + x[:, 9],
        -8*x[:, 1] + x[:, 10],
        -8*x[:, 2] + x[:, 11],
        -2*x[:, 3] - x[:, 4] + x[:, 9],
        -2*x[:, 5] - x[:, 6] + x[:, 10],
        -2*x[:, 7] - x[:, 8] + x[:, 11],
    ], 1)
    c[:, 1:] = np.maximum(c[:, 1:], 0.)
    return c
    

def g6_objective(x):
    c = np.stack([
        # objective
        (x[:, 0] - 10.)**3 + (x[:, 1] - 20.)**3,
        # constraints
        -(x[:, 0] - 5)**2 - (x[:, 1]-5)**2 + 100,
        (x[:, 0] - 6)**2 + (x[:, 1]-5)**2 - 82.81,
    ], 1)
    c[:, 1:] = np.maximum(c[:, 1:], 0.)
    return c


def g2_objective(x):
    n, d = x.shape
    c = np.cos(x)
    r = np.stack([
        # objective
        -np.abs(((c**4).sum(1) - 2*(c**2).prod(1)) / np.sqrt((np.arange(d) * x**2).sum(1))),
        # constraints
        0.75 - x.prod(1),
        x.sum(1) - 7.5*d
    ], 1)
    r[:, 1:] = np.maximum(r[:, 1:], 0.)
    return r

In [4]:
objective = g6_objective
n_constraints = 2

x_min = np.array([13., 0.])
x_max = 100.
d = 2
n = 200
alpha_inf = 0.2
eta_c = 15.
eta_m = 20.
p_c = 0.8
p_m = 0.1
num_iterations = 1750

populations, scores = IDEA(objective, n_constraints, x_min, x_max, d, n, alpha_inf, eta_c, eta_m, p_c, p_m, num_iterations, log_interval=100)

Iteration 0, #feasible: 0, best: -, #infeasible: 200, best: [-5.51746004e+03  1.00000000e+00]
Iteration 100, #feasible: 160, best: [-6433.09402266     0.        ], #infeasible: 40, best: [-7.92675379e+03  1.00000000e+00]
Iteration 200, #feasible: 160, best: [-6960.20007846     0.        ], #infeasible: 40, best: [-7.973e+03  1.000e+00]
Iteration 300, #feasible: 160, best: [-6960.53360921     0.        ], #infeasible: 40, best: [-7.973e+03  1.000e+00]
Iteration 400, #feasible: 160, best: [-6960.53877641     0.        ], #infeasible: 40, best: [-7.973e+03  1.000e+00]
Iteration 500, #feasible: 160, best: [-6960.53877641     0.        ], #infeasible: 40, best: [-7.973e+03  1.000e+00]
Iteration 600, #feasible: 160, best: [-6960.53877641     0.        ], #infeasible: 40, best: [-7.973e+03  1.000e+00]
Iteration 700, #feasible: 160, best: [-6960.69921367     0.        ], #infeasible: 40, best: [-7.973e+03  1.000e+00]
Iteration 800, #feasible: 160, best: [-6960.69921367     0.        ], #infeas

In [5]:
scores[-1, scores[-1, :, 1] == 0., 0].min()

-6961.046339188843

In [6]:
objective = g1_objective
n_constraints = 9

x_min = 0.
x_max = np.ones(13)
x_max[9:12] = 100.
d = 13
n = 200
alpha_inf = 0.2
eta_c = 15.
eta_m = 20.
p_c = 0.8
p_m = 0.1
num_iterations = 1750

populations, scores = IDEA(objective, n_constraints, x_min, x_max, d, n, alpha_inf, eta_c, eta_m, p_c, p_m, num_iterations, log_interval=100)

Iteration 0, #feasible: 0, best: -, #infeasible: 200, best: [-257.72691377   36.        ]
Iteration 100, #feasible: 160, best: [-4.65003975  0.        ], #infeasible: 40, best: [-298.96681856   47.        ]
Iteration 200, #feasible: 160, best: [-4.81654865  0.        ], #infeasible: 40, best: [-299.58722332   14.        ]
Iteration 300, #feasible: 160, best: [-5.21838103  0.        ], #infeasible: 40, best: [-299.60798895    1.        ]
Iteration 400, #feasible: 160, best: [-6.16443832  0.        ], #infeasible: 40, best: [-299.75224227    5.        ]
Iteration 500, #feasible: 160, best: [-7.08530938  0.        ], #infeasible: 40, best: [-299.89482539    1.        ]
Iteration 600, #feasible: 160, best: [-7.49067745  0.        ], #infeasible: 40, best: [-300.24325182    1.        ]
Iteration 700, #feasible: 160, best: [-7.56972798  0.        ], #infeasible: 40, best: [-300.29680677    1.        ]
Iteration 800, #feasible: 160, best: [-7.57350985  0.        ], #infeasible: 40, best: [-30

In [7]:
scores[-1, scores[-1, :, 1] == 0., 0].min()

-8.933862224129514

In [8]:
objective = g2_objective
n_constraints = 2

x_min = 1e-4
x_max = 10.
d = 20
n = 200
alpha_inf = 0.2
eta_c = 15.
eta_m = 20.
p_c = 0.8
p_m = 0.1
num_iterations = 1750

populations, scores = IDEA(objective, n_constraints, x_min, x_max, d, n, alpha_inf, eta_c, eta_m, p_c, p_m, num_iterations, log_interval=100)

Iteration 0, #feasible: 199, best: [-0.22392464  0.        ], #infeasible: 1, best: [-0.07480228  1.        ]
Iteration 100, #feasible: 160, best: [-0.39225704  0.        ], #infeasible: 40, best: [-0.35907293  1.        ]
Iteration 200, #feasible: 160, best: [-0.41232662  0.        ], #infeasible: 40, best: [-0.38357035  1.        ]
Iteration 300, #feasible: 160, best: [-0.42087605  0.        ], #infeasible: 40, best: [-0.40450642  1.        ]
Iteration 400, #feasible: 160, best: [-0.45890818  0.        ], #infeasible: 40, best: [-0.43035665  1.        ]
Iteration 500, #feasible: 160, best: [-0.48130251  0.        ], #infeasible: 40, best: [-0.44690138  1.        ]
Iteration 600, #feasible: 160, best: [-0.48458948  0.        ], #infeasible: 40, best: [-0.44871362  1.        ]
Iteration 700, #feasible: 160, best: [-0.48615225  0.        ], #infeasible: 40, best: [-0.4497726  1.       ]
Iteration 800, #feasible: 160, best: [-0.48779075  0.        ], #infeasible: 40, best: [-0.4507673  1

In [9]:
scores[-1, scores[-1, :, 1] == 0., 0].min()

-0.4964954473478704

Sanity check

In [10]:
i = scores[-1, scores[-1, :, 1] == 0., 0].argmin()
i

0

In [13]:
x = populations[-1, scores[-1, :, 1] == 0., :][[i], :]

In [14]:
objective(x)

array([[-0.49649545,  0.        ,  0.        ]])